In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sqlite3 import Error
from sklearn.ensemble import RandomForestClassifier
import sqlite3
import pickle
%matplotlib inline
try:
    conn = sqlite3.connect("training_V2.db")
except Error as e:
    print(e)
# df1.to_excel(r'C:\Users\Antonio Fonseca\Desktop\BLOX\ementas.xlsx')
# n esquecer do tensorflow

In [10]:
df = pd.read_excel('atual.xlsx')
df.shape

(4633, 2)

In [11]:
df["ÁREA DO CONHECIMENTO"].value_counts()

Engenharia, Produção e Construção            654
Ciências Naturais                            588
Saúde e Bem-Estar Social                     587
Ciências Sociais, Jornalismo e Informação    509
Humanidades e Linguagens                     385
Matemática e Computação                      333
Negócios e Administração                     321
Direito                                      279
Agricultura e Veterinária                    246
Educação                                     200
programas genericos                          161
Serviços                                     142
Psicologia e Desenvolvimento                 137
Programas Interdisciplinares                  91
Name: ÁREA DO CONHECIMENTO, dtype: int64

In [12]:
data = pd.read_excel('ementas.xlsx')
data.shape

(10165, 7)

In [13]:
data.head()

,Unnamed: 0,materias,escola,obj,p_r,programa,ÁREA DO CONHECIMENTO
0,0,"Práticas Multidisciplinares em Canto Coral, co...",Escola de Comunicações e Artes,1- Oferecer aos alunos do conjunto de discipli...,1.Estágio no Coral Escola Comunicantus e no Co...,1.Estágio no Coral Escola Comunicantus e no Co...,NaN
1,1,Introdução à Computação II,"Faculdade de Filosofia, Ciências e Letras de R...",Introduzir os conceitos básicos de projeto e a...,Análise de complexidade de algoritmos; Paradig...,NaN,NaN
2,2,Análise Ambiental: Gestão de Áreas Protegidas,Escola de Engenharia de São Carlos,Discutir a importância do estabelecimento de e...,"1. Apresentação da disciplina, do professor e ...","1. Apresentação da disciplina, do professor e ...",NaN
3,3,Práticas Interpretativas III,"Faculdade de Filosofia, Ciências e Letras de R...",Introduzir aos alunos de instrumento uma noção...,Estudar os estilos de interpretação dentro do ...,– Estudar os estilos de interpretação dentro d...,NaN
4,4,Ações Comunitárias I,Instituto de Psicologia,"Exposição do aluno a situações comunitárias, s...","Exposição do aluno a situações comunitárias, s...","""\tExperiência de observação ou observação par...",NaN


In [14]:
data = data.sort_values('materias', ascending=True)
data.head()

,Unnamed: 0,materias,escola,obj,p_r,programa,ÁREA DO CONHECIMENTO
7,7,"""Arte dos anos 1960 à atualidade: explorando f...",Escola de Comunicações e Artes,1. Oferecer aos estudantes um quadro geral das...,Discussão de temas previamente propostos em se...,1. A tradição humanista europeia posta em xequ...,NaN
6487,6487,(Des) Construção de traje de cena e cenografi...,Escola de Comunicações e Artes,Discutir o traje de cena contemporâneo a parti...,O traje de cena popular – origens e trajetória...,1-O traje de cena popular – origens e trajetór...,NaN
1790,1790,(Des) Construção de traje de cena e cenografia...,Escola de Comunicações e Artes,Traçar amplo panorama da inserção do traje de ...,"A cena queer\nÉ da Disney, sim, e daí? Cenogra...",1- Breve histórico do traje de cena e cenograf...,NaN
2040,2040,A Abordagem Qualitativa na Pesquisa em Psicolo...,Instituto de Psicologia,Pretende fornecer elementos para que os alunos...,NaN,1. Introdução à abordagem qualitativa na pesqu...,NaN
3433,3433,A Cidade Constitucional,"Escola de Artes, Ciências e Humanidades",A Cidade Constitucional é um projeto de extens...,Disciplina de campo ligada ao Direito Constitu...,Conteúdo: Aulas teóricas e práticas\n\nAs aula...,NaN


In [15]:
teste = df.merge(data, on= ["materias"],how ="inner" )

In [17]:
teste.head()

,materias,ÁREA DO CONHECIMENTO_x,Unnamed: 0,escola,obj,p_r,programa,ÁREA DO CONHECIMENTO_y
0,Acolhimento em Emergências,programas genericos,3278,Faculdade de Medicina de Ribeirão Preto,1. Compreender os desafios ético-sociais envol...,O Objetivo geral é que o aluno tenha noção do ...,Atividades Teóricas: \n1.Macro e Micro regulaç...,NaN
1,Apoio à Escolha da Especialidade Médica,programas genericos,1859,Faculdade de Medicina de Ribeirão Preto,Compreender os principais fatores que influenc...,Estudo de fatores ligados ao estudante e à nat...,Fatores que influenciam a escolha da especiali...,NaN
2,"Atividades Acadêmicas Científicas, de Extensão...",programas genericos,5117,Instituto de Ciências Matemáticas e de Computação,"Estimular a pró-atividade dos alunos, incentiv...",São computadas as participações em: atividades...,1. Atividades de extensão: cursos de difusão c...,NaN
3,"Atividades Acadêmicas Científicas, de Extensão...",programas genericos,8285,Instituto de Ciências Matemáticas e de Computação,"Estimular a pró-atividade dos alunos, incentiv...",São computadas as participações em: atividades...,1. Atividades de extensão: cursos de difusão c...,NaN
4,Atividades Acadêmico-Científico-Culturais I (A...,programas genericos,176,Pró-Reitoria de Graduação - Cursos Interunidades,NaN,NaN,NaN,NaN


In [19]:
teste = teste.drop(['Unnamed: 0', 'escola','ÁREA DO CONHECIMENTO_y'], axis=1)

In [26]:
teste.to_csv(r'C:\Users\Antonio Fonseca\Desktop\BLOX\treino.csv')